In [1]:
from __future__ import print_function
from keras.engine import Model
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from pre_processing import load_data

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
(X_train, y_train), (X_test, y_test), vocab = load_data()

In [12]:
maxlen = 0
for a in X_train:
    if a.shape[0] > maxlen: maxlen = a.shape[0]
maxlen

15

In [13]:
vgg_model = VGGFace(include_top=False, pooling='avg')

In [14]:
features_train = []
for word_frames in X_train:
    features_train.append(vgg_model.predict(word_frames))
    
features_test = []
for word_frames in X_test:
    features_test.append(vgg_model.predict(word_frames))

In [15]:
# pad features, figure this number out beforehand
for i, f in enumerate(features_train):
    # grab last set of features and duplicate it
    last = np.array(f[-1].reshape(1,512))
    for _ in range(f.shape[0], maxlen):
        features_train[i] = np.concatenate((features_train[i],last), axis=0)
        
for i, f in enumerate(features_test):
    # grab last set of features and duplicate it
    last = np.array(f[-1].reshape(1,512))
    for _ in range(f.shape[0], maxlen):
        features_test[i] = np.concatenate((features_test[i],last), axis=0)

In [16]:
# numpy array
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [17]:
features_train = np.asarray(features_train)
features_test = np.asarray(features_test)

In [18]:
model = Sequential()
model.add(LSTM(1000, input_shape=(maxlen, 512)))
model.add(Dense(1, activation='tanh'))

In [19]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(features_train, y_train, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
424/424 [==============================] - 6s - loss: 494.9458 - acc: 0.0377     

In [23]:
score, acc = model.evaluate(features_test, y_test, batch_size=32)
print("Test score: ", score)
print("Test acc: ", acc)

183/183 [==============================] - 0s     
Test score:  485.28415534
Test acc:  0.0655737706954
